In [1]:
from sweetbean import Block, Experiment
from sweetbean.stimulus import Bandit, Text
from sweetbean.variable import (
    DataVariable,
    FunctionVariable,
    SharedVariable,
    SideEffect,
    TimelineVariable,
)

In [13]:
!pip install --upgrade sweetbean==1.3.3

^C


In [2]:
import json

# Welcome message
instruction_welcome = Text(
    text='Welcome to our decision-making experiment. In this experiment, you will make decisions between two options. \
    Press the SPACE key to continue.',
    choices=[' ']
)

# Explanation of the task
instruction_task = Text(
    text=json.dumps("""
    <div class="slotmachine" style="position: absolute; top:1vh; left:0 vw; width: 20vw; height: 20vh; border-color:orange
        ">
        </div>
        <div class="slotmachine" style="position: absolute; top:1vh; right:0 vw; left:40vw; width: 20vw; height: 20vh; border-color:blue
        ">
        </div>
    """)+ 'In each trial, you will see two slot machines (bandits) on the screen. <b> Click on a bandit </b> to make your choice\
    Press SPACE to continue.',
    choices=[' ']
)

# Explanation of feedback/reward system
instruction_feedback = Text(
    text='After selecting a bandit, you will receive feedback showing whether you won reward (1 point) or not (0 point). One bandit tends to give rewards more often than the other. However, this may change during the experiment.\
    Press SPACE to continue.',
    choices=[' ']
)

# End instructions
instruction_end = Text(
    text='Your goal is to earn as many points as possible. Pay attention to feedback, as the probabilities of winning for each bandit may change over time. \
    You are now ready to begin. Good luck! Press Q to start the task.',
    choices=['q', ' ']
)

# Create a list of instruction 
# for the instruction block
instruction_list = [
    instruction_welcome,
    instruction_task,
    instruction_feedback,
    instruction_end
]

# Create the instruction block
instruction_block = Block(instruction_list)

In [6]:
import random

random.seed(15)

# Number of trials
num_trials = 10

# Define the timeline
timeline = []

for i in range(num_trials):
    # Determine if bandits should switch
    if i < (num_trials/2):
        # Determine rewards based on probabilities
        bandit_1_reward = random.choices(
            [1, 0], weights=[0.8, 0.2]
        )[0]  # 80% success rate

        bandit_2_reward = random.choices(
            [1, 0], weights=[0.2, 0.8]
        )[0]  # 20% success rate

    else:
      # Determine rewards based on probabilities
        bandit_1_reward = random.choices(
            [1, 0], weights=[0.2, 0.8]
        )[0]  # 80% success rate

        bandit_2_reward = random.choices(
            [1, 0], weights=[0.8, 0.2]
        )[0]  # 20% success rate
    # Append the trial to the timeline
    timeline.append({
        "bandit_1": {"color": "orange", "value": bandit_1_reward},
        "bandit_2": {"color": "blue", "value": bandit_2_reward}
    })

# Print the timeline (for testing)
for i, trial in enumerate(timeline):
    print(f"Trial {i+1}: {trial}")

Trial 1: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 2: {'bandit_1': {'color': 'orange', 'value': 1}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 3: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 4: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 0}}
Trial 5: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 0}}
Trial 6: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 7: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 8: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}
Trial 9: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 0}}
Trial 10: {'bandit_1': {'color': 'orange', 'value': 0}, 'bandit_2': {'color': 'blue', 'value': 1}}


In [7]:
bandit_1 = TimelineVariable("bandit_1")
bandit_2 = TimelineVariable("bandit_2")

score = SharedVariable("score", 0)
value = DataVariable("value", 0)

update_score = FunctionVariable(
    "update_score", lambda sc, val: sc + val, [score, value]
)

update_score_side_effect = SideEffect(score, update_score)

bandit_task = Bandit(
    bandits=[bandit_1, bandit_2],
    side_effects=[update_score_side_effect],
)

score_text = FunctionVariable("score_text", lambda sc: f"Score: {sc}", [score])

show_score = Text(duration=1000, text=score_text)

trial_sequence = Block([bandit_task, show_score], timeline=timeline)
experiment = Experiment([instruction_block,trial_sequence])

In [8]:
experiment.to_html("bandit_new.html", path_local_download="2armed_bandit.json")